In [50]:
def append(hash, byte, pol, poly_ring):
    hash = hash * poly_ring(256.digits(2)) # << 8
    hash = hash + poly_ring(byte.digits(2)) # | Pol(b)
    
    return hash % pol

In [194]:
def orPoly(x, y, poly_ring):
    return poly_ring([a or b for (a, b) in zip(x.list(), y.list())])

# https://github.com/restic/chunker/blob/master/chunker.go#L162
def generateTable(irr, poly_ring, deg, window_size):
    out = []
    for i in range(0, 256):
        h = poly_ring(0)

        h = append(h, Integer(i), irr, poly_ring)
        for j in range(0, window_size - 1):
            h = append(h, 0, irr, poly_ring)
        out.append(h)
        
    mod = []
    k = irr.degree()
    
    for i in range(0, 256):
        mod.append(orPoly(poly_ring(Integer(i << k % deg).digits()) % irr, poly_ring(Integer(i << k % deg).digits()), poly_ring))
    
    
    return (out, mod)

In [285]:
def byteToInt(poly):
    if poly.degree() < 24:
        return 0
    return ZZ(poly.list()[24:32], base = 2)
    
    
    return 
def updateDigest (digest, tables, byte, poly_ring):
    index = byteToInt(digest) # get top byte
    digest = (digest * poly_ring(256.digits(2))) % irr # << 8
    digest = (digest + poly_ring(Integer(byte).digits(2))) % irr # | Pol(b)
    
    digest = (digest + tables[1][index]) % irr
    return digest

In [300]:
def slide (digest, byte, tables, state, poly_ring):
    out = state.window[state.wpos]
    state.window[state.wpos] = byte
    digest = (digest + tables[0][out]) % irr
    state.wpos = (state.wpos + 1) % state.windowSize
    
    digest = updateDigest(digest, tables, byte, poly_ring)
    return digest % irr

In [397]:
class Chunker:
    def __init__(self, poly_ring, windowSize, tables, irr):
        self.poly_ring = poly_ring
        self.windowSize = windowSize
        self.tables = tables
        self.irr = irr
        self.reset()
        
    def reset(self):
        self.window = []
        for i in range(0, self.windowSize):
            self.window.append(0)
        self.wpos = 0
        self.digest = self.poly_ring(0)
        self.digest = slide(self.digest, 1, self.tables, self, self.poly_ring)

In [398]:
from sage.crypto.util import ascii_to_bin
f = open("/tmp/files/robinson-crusoe-missing.txt", "rb")
bytes = bytearray(f.read())

In [399]:
P = polygen(GF(2), 'x')
poly_ring = P.parent()
irr = poly_ring([1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [400]:
tables = generateTable(irr, poly_ring, 32, 64)

In [ ]:
state = Chunker(poly_ring, 64, tables, irr)
mask = (1 << 13) - 1
for i, byte in enumerate(bytes):
    state.digest = slide(state.digest, byte, tables, state, poly_ring)
    
    asInt = ZZ(state.digest.list(), base = 2)
    if asInt & mask == 0:
        print("{0}\t{1}\t{2}".format(i % 256, i, asInt))
        state.reset()

/Applications/SageMath-8.7.app/Contents/Resources/sage/local/lib/python2.7/site-packages/sage/repl/ipython_kernel/__main__.py:7: DeprecationWarning: polynomial slicing with a start index is deprecated, use list() and slice the resulting list instead
See http://trac.sagemath.org/18940 for details.


113	20081	935280640
126	22910	1204920320
113	31345	209068032
20	33044	1538301952
199	37319	742105088
187	39867	569729024
163	42659	23691264
221	49373	858734592
177	49841	1071874048
176	61360	803405824
198	65478	1052033024
238	70126	1127849984
226	72162	602112000
154	80026	792895488
204	83404	1833402368
213	91093	2098339840
213	100565	1243496448
16	105232	898908160
22	118038	1537122304
48	123440	1552015360
5	140549	766099456
109	143469	990445568
144	148624	1251508224
104	160872	245178368
52	168500	11657216
178	172722	1227710464
99	185187	919871488
102	196454	424452096
135	205447	512737280
145	217489	1748434944
105	220009	296116224
3	223235	1269809152
34	238882	1734541312
41	240681	1643872256
23	244503	893329408
46	251438	1262567424
174	264366	1420402688
231	268007	723296256
46	275758	803971072
110	294254	2136432640
181	295349	31875072
54	298806	1235337216
222	303582	1487577088
100	304740	1170046976
165	313253	1674264576
44	322604	888119296
158	328094	1449517056
29	329757	21643264
4	3338